In [237]:
!pip install dash

In [238]:
import pandas as pd
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/team/2"

headers = {
	"X-RapidAPI-Key": "24a37464fbmsh394f776e093a977p1e050fjsna4fc3547c5c3",
	"X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
}
params = {
    "teamId": 2,
    "statsType":"mostRuns",
    "year":2023,
    "matchType":1
}

response = requests.get(url, headers=headers,params=params)

print(response.json()['values'])

[{'values': ['1413', 'Kohli', '8', '12', '671', '55.92']}, {'values': ['576', 'Rohit', '8', '13', '545', '41.92']}, {'values': ['13940', 'Yashasvi Jaiswal', '3', '5', '288', '57.60']}, {'values': ['587', 'Ravindra Jadeja', '7', '9', '281', '35.12']}, {'values': ['8808', 'Axar', '4', '5', '264', '88.00']}, {'values': ['11808', 'Shubman Gill', '6', '10', '258', '28.67']}, {'values': ['1448', 'Pujara', '5', '8', '181', '25.86']}, {'values': ['1593', 'Ashwin', '7', '8', '150', '18.75']}, {'values': ['1447', 'Ajinkya Rahane', '3', '4', '146', '36.50']}, {'values': ['8733', 'Rahul', '3', '5', '143', '28.60']}, {'values': ['9025', 'Srikar Bharat', '5', '8', '129', '18.43']}, {'values': ['9428', 'Shreyas Iyer', '4', '6', '79', '13.17']}, {'values': ['10276', 'Ishan Kishan', '2', '3', '78', '78.00']}, {'values': ['8683', 'Thakur', '3', '4', '77', '19.25']}, {'values': ['7909', 'Shami', '4', '5', '65', '21.67']}, {'values': ['1858', 'Umesh', '3', '5', '23', '4.60']}, {'values': ['10808', 'Siraj'

In [239]:
json_data=response.json()['values']

values_list = [entry['values'] for entry in json_data]

columns = ['PlayerID', 'PlayerName', 'MatchesPlayed', 'Innings', 'Runs', 'Average']

df = pd.DataFrame(values_list, columns=columns)

In [240]:
params_2 = {
    "teamId": 2,
    "statsType":"mostRuns",
    "year":2023,
    "matchType":2
}

response_2 = requests.get(url, headers=headers,params=params_2)

json_data_2=response_2.json()['values']

values_list_2 = [entry['values'] for entry in json_data_2]

columns = ['PlayerID', 'PlayerName', 'MatchesPlayed', 'Innings', 'Runs', 'Average']

df_2 = pd.DataFrame(values_list_2, columns=columns)

In [241]:
params_3 = {
    "teamId": 2,
    "statsType":"mostRuns",
    "year":2023,
    "matchType":3
}

response_3 = requests.get(url, headers=headers,params=params_3)

json_data_3=response_3.json()['values']

values_list_3 = [entry['values'] for entry in json_data_3]

columns = ['PlayerID', 'PlayerName', 'MatchesPlayed', 'Innings', 'Runs', 'Average']

df_3 = pd.DataFrame(values_list_3, columns=columns)

In [242]:
common_ids_df = pd.merge(df, pd.merge(df_2, df_3, on='PlayerID'), on='PlayerID')

In [243]:
common_df = common_ids_df[['PlayerName_x', 'Average', 'Average_x', 'Average_y','Runs','Runs_x','Runs_y','MatchesPlayed','MatchesPlayed_x','MatchesPlayed_y']].copy()

common_df.columns = ['PlayerName', 'Test Average', 'ODI Average', 'T20 Average','Test Runs','ODI Runs','T20 Runs','Test Matches Played','ODI Matches Played','T20 Matches Played']

common_df

,PlayerName,Test Average,ODI Average,T20 Average,Test Runs,ODI Runs,T20 Runs,Test Matches Played,ODI Matches Played,T20 Matches Played
0,Ravindra Jadeja,35.12,30.90,11.50,281,309,23,7,26,2
1,Axar,88.00,17.50,27.14,264,140,190,4,11,13
2,Shubman Gill,28.67,63.36,26.00,258,1584,312,6,29,13
3,Shreyas Iyer,13.17,52.88,30.50,79,846,61,4,20,2
4,Ishan Kishan,78.00,35.08,18.82,78,456,207,2,17,11
5,Suryakumar Yadav,8.00,21.61,48.87,8,389,733,1,21,18


In [245]:
from dash import Dash,dcc,html,Input,Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Indian Cricket Players DashBoard'),
    html.Div([
        dcc.Dropdown(
            id='format',
            value=1,
            options=[{'label': 'Test', 'value': 1},{'label': 'ODI', 'value': 2},{'label':'T20','value':3}]
                  ),
        dcc.RadioItems(
            id='year',
            value=2020,
            options=[{'label': '2020', 'value': 2020},{'label': '2021', 'value': 2021},{'label':'2022','value':2022},{'label':'2023','value':2023}]
                  ),

        html.Div(html.H4(children='Indian Players Runs Scored:')),
        html.Div(id='table-title'),
        html.Div(id='table-output')
        ]),
    html.Div([
        html.H3(children="Statistics Of Players who played in All Three Format in the year of 2023"),
        dcc.Dropdown(id='feature',
                 value='Average',
                 options=[{'label': 'Average', 'value': 'Average'},{'label': 'Runs', 'value': 'Runs'},{'label':'Matches Played','value':'Matches Played'}]
                     ),
        html.Div(
            dcc.Graph(id='bar-graph'))
        ]),
    html.Div([
      dcc.RadioItems(
            id='player-id',
            value='265',
            options=[{'label':'MS Dhoni','value':'265'},{'label':'Rahul Dravid','value':'27'},{'label':'Sachin Tendulkar','value':'25'},{'label':'Suresh Raina','value':'413'},{'label': 'Virat Kohli', 'value': '1413'},{'label': 'Rohit Sharma', 'value': '573'},{'label':'KL Rahul','value':'8733'},{'label':'Ravindra Jadeja','value':'587'}]
                  ),
      dcc.Dropdown(
        id='stat-dropdown',
        options=[{'label': stat, 'value': stat} for stat in ['Matches','Innings','Runs','Highest','Average','SR','Not Out']],
        value='Matches'
    ),
    dcc.Graph(id='stat-graph')
      ])
])

@app.callback(
    [Output(component_id='table-title',component_property='children'),
     Output(component_id='table-output',component_property='children')],
    [Input(component_id='format',component_property='value'),
     Input(component_id='year',component_property='value')]
)
def update_table(format,year):
  format_title=['Test','ODI','T20']
  title= f"Top Runs Scorer\'s in {format_title[format-1]} in the year of {year}"
  params= {
    "teamId": 2,
    "statsType":"mostRuns",
    "year":year,
    "matchType":format
    }
  response = requests.get(url, headers=headers,params=params)
  json_data=response.json()['values']
  values_list= [entry['values'] for entry in json_data]
  columns = ['PlayerID', 'PlayerName', 'MatchesPlayed', 'Innings', 'Runs', 'Average']
  df= pd.DataFrame(values_list, columns=columns)

  table=html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in df.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(df.iloc[i][col]) for col in df.columns
            ]) for i in range(min(len(df), 20))
        ])
    ])
  return title,table

@app.callback(
    Output(component_id='bar-graph', component_property='figure'),
    [Input(component_id='feature', component_property='value')],
)
def update_graph(value):
  desired_format = {
    'PlayerName': common_df['PlayerName'].values,
    'Test '+value: common_df['Test '+value].values.astype(float),
    'ODI '+value: common_df['ODI '+value].values.astype(float),
    'T20 '+value: common_df['T20 '+value].values.astype(float)
}
  plot_df = pd.DataFrame(desired_format)

  fig = px.bar(plot_df, x='PlayerName', y=['Test '+value, 'ODI '+value, 'T20 '+value],
             title='Player '+ value + ' Across Formats', labels={'PlayerName': 'Players', 'value': value},
             barmode='group')
  fig.update_layout(legend_title='Formats')
  return fig


@app.callback(
    Output('stat-graph', 'figure'),
    [Input('player-id', 'value'),Input('stat-dropdown', 'value')]
)
def update_graph_2(playerid,selected_stat):
  url="https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/"+playerid+"/batting"
  response=requests.get(url, headers=headers)
  player=response.json()['appIndex']['seoTitle']
  header = response.json()['headers']
  values = [row['values'] for row in response.json()['values']]
  df = pd.DataFrame(values, columns=header)

  desired_format={
    'Format':['Test','ODI','T20','IPL'],
    'Value':[int(df[df['ROWHEADER'] == selected_stat]['Test'].iloc[0]),int(df[df['ROWHEADER'] == selected_stat]['ODI'].iloc[0]),int(df[df['ROWHEADER'] == selected_stat]['T20'].iloc[0]),int(df[df['ROWHEADER'] == selected_stat]['IPL'].iloc[0])]}
  plot_df=pd.DataFrame(desired_format)
  fig = px.bar(plot_df, x=['Test','ODI','T20','IPL'], y='Value',
             title=player[:-34]+' Statistics Across Formats', labels={'x':'Format','Value':selected_stat},
             barmode='group')
  return fig




if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>